In [54]:
import os.path

import pandas as pd
import glob
import os

dir = 'C:/Users/liams/Pictures/capstoneGIStiff/processed'

files = glob.glob('C:/Users/liams/Pictures/capstoneGIStiff/processed' + '/*.csv')

dataframes = []

for path in files:
    df = pd.read_csv(path).rename(columns={'norm_value': os.path.basename(path)})
    df['long'] = df['long'].round(4)
    df['lat'] = df['lat'].round(4)
    df = df.set_index(['long', 'lat'])
    dataframes.append(df)

frame = pd.concat(dataframes, axis=1).fillna(0)

In [55]:
frame

,,WA_red_alder_stand_density.csv,WA_sitka_spruce_stand_density.csv
long,lat,,
-122.6325,49.0010,0.218944,0.000000
-122.6074,49.0010,0.187888,0.000000
-122.1345,49.0010,0.001553,0.000000
-122.1342,49.0010,0.001553,0.000000
-123.0655,49.0009,0.068323,0.000000
...,...,...,...
-123.7689,45.5719,0.000000,0.189655
-123.7686,45.5719,0.000000,0.189655
-123.7209,45.5719,0.000000,0.018678
